# 经典Benders分解
## 1. 算法原理
**Benders分解**是行生成方法即**割平面法**的一种。割平面法的思想是，确定最优解可能不需要全部约束，因此可以逐行增加约束（割平面），减小可行域。

经典Benders分解针对**混合整数线性规划（MILP）**问题。对于一个MILP原问题，将其分解为**主问题**和**子问题**，并将原问题的变量分给这两个问题求解，一般将复杂变量或整数变量放在主问题中，将连续变量放在子问题中。先求解主问题，得到原问题下界，并将解传递给子问题。之后求解**对偶子问题**，若对偶子问题**有最优解**，则也得到子问题最优解，更新原问题上界，生成最优性割。若对偶子问题**无界**，则子问题无解，生成可行性割。将最优性割或可行性割加到主问题中，求解主问题并更新下界，迭代直到上下界很小为止。


## 2. 问题拆解
### 2.1 原问题：

原问题是一个最小化MILP问题，$\mathbf{x}$为整数变量，$\mathbf{y}$为连续变量，$\mathbf{x}$、$\mathbf{y}$均为向量。
$$
\begin{aligned}
\min \quad & \mathbf{c}^\top \mathbf{x} + \mathbf{d}^\top \mathbf{y} \\
\text{s.t.} \quad & \mathbf{A} \mathbf{x} + \mathbf{B} \mathbf{y} \ge \mathbf{b} \\
& \mathbf{y} \ge 0 \\
& \mathbf{x} \in \mathbb{X}
\end{aligned}
$$


### 2.2 主问题：

主问题的变量是原问题的**整数变量**$\mathbf{x}$，目标函数与原问题相同。主问题不包含原问题的约束条件，因此是**松弛主问题**。求解松弛主问题，得到原问题的一个下界。主问题中， $\theta$是辅助变量，用来“占位”表示“对子问题目标函数$\phi(x)$的上界估计”，通过不断加割把$\theta$收紧，最终逼得$\theta=\phi(x)$。
$$
\begin{aligned}
\min \quad & \mathbf{c}^\top \mathbf{x} +  \theta \\
\text{s.t.} \quad & \mathbf{x} \in \mathbb{X}
\end{aligned}
$$


### 2.3 子问题：

将松弛主问题的求解结果带入子问题。子问题的变量是原问题的**连续变量**$\mathbf{y}$，目标函数$\phi(\mathbf{x})=\mathbf{d}^\top \mathbf{y}$。
$$
\begin{aligned}
\phi(\mathbf{x}) = \min \quad & \mathbf{d}^\top \mathbf{y} \\
\text{s.t.} \quad & \mathbf{B} \mathbf{y} \ge \mathbf{b} - \mathbf{A} \bar{\mathbf{x}} \\
& \mathbf{y} \ge 0
\end{aligned}
$$


### 2.4 对偶子问题：
在经典Benders分解中，取对偶的前提是子问题是一个**LP问题**。LP问题具有**强对偶性**，若对偶子问题有最优解，则对偶子问题最优解也必定是子问题最优解。

子问题目标函数是最小化，因此对偶问题目标函数是最大化：
$$
\begin{aligned}
\max \quad & (\mathbf{b} - \mathbf{A} \bar{\mathbf{x}})^\top \mathbf{u} \\
\text{s.t.} \quad & \mathbf{B}^\top \mathbf{u} \le \mathbf{d} \\
& \mathbf{u} \ge 0
\end{aligned}
$$



## 3. 可行性割与最优性割：
### 3.1 可行性割：

我们先引入**极射线**的概念。若对偶子问题**无界**（子问题无解），则其可行域存在某个方向（称为极射线），在这个方向上目标函数可以无限增加。极射线本质上是一种“逃离有界区域的方向”，描述了约束系统失效的方向性证据。我们利用这个极射线，构造一个**可行性割（feasibility cut）**，这个割将被引入主问题，以排除那些会导致子问题不可行的$\mathbf{x}$值：
$$
\mathbf{r}_t^\top (\mathbf{b} - \mathbf{A}\mathbf{x}) \le 0 \quad \text{for } t = 1, 2, \dots, T
$$



### 3.2 最优性割

若对偶子问题在当前主问题解 $\bar{\mathbf{x}}$ 下有最优解，则说明子问题在该 $\bar{\mathbf{x}}$ 下是可行并有界的。此时，我们可以从中获得子问题的一个最优解 $\mathbf{y}^\ast$，从而构造出原问题的一个可行解 $(\bar{\mathbf{x}}, \mathbf{y}^\ast)$，其为原问题提供了一个**上界**，可用于更新当前原问题的最优值。

继续利用对偶理论，我们还可以从对偶子问题的最优对偶解 $\mathbf{u}^\ast$ 中提取关于子问题最优值函数 $\phi(x)$ 的**线性下界**：

$$
\phi(x) \ge (\mathbf{b} - \mathbf{A} \mathbf{x})^\top \mathbf{u}^\ast
$$

为在主问题中逼近 $\phi(x)$，我们令主问题中的辅助变量 $\theta$ 代表对子问题最优值的估计，并引入如下**最优性割（optimality cut）**，保证 $\theta$ 永远不小于子问题的真实最优值，从而保持目标函数的正确性（这里推导比较复杂）：

$$
\theta \ge (\mathbf{b} - \mathbf{A} \mathbf{x})^\top \mathbf{u}^\ast
$$



## 4 迭代流程

### 4.1 初始化
* 选取一个可行或启发式起点 $x^{(0)}$  
* 设上下界  

   $$
   \mathrm{UB}\leftarrow+\infty,\qquad
   \mathrm{LB}\leftarrow-\infty,\qquad
   k\leftarrow0
   $$


### 4.2 循环迭代

#### 4.2.1 子问题  
$$
\phi\!\bigl(x^{(k)}\bigr)=
\min_{y\ge0}\Bigl\{\,d^{\top}y \;\bigm|\; By \ge b-Ax^{(k)}\Bigr\}
$$

* **若不可行**：取无界射线 $r_t$，生成可行性割  
  $$
  r_t^{\top}(b-Ax)\le 0
  $$

* **若可行且有界**：得对偶最优解 $u_s^{\star}$，生成最优性割，并更新上界  

  $$
  \theta \;\ge\; (b-Ax)^{\top}u_s^{\star}
  $$

  $$
  \mathrm{UB}\;\leftarrow\;
  \min\!\Bigl\{\,\mathrm{UB},\;c^{\top}x^{(k)}+\phi\!\bigl(x^{(k)}\bigr)\Bigr\}
  $$

#### 4.2.2 主问题（累积所有割）  
$$
\min_{x\in\mathbb X,\;\theta}\; c^{\top}x + \theta\\
\text{s.t.已生成的全部可行性割与最优性割}
$$

得到新解 $\bigl(x^{(k+1)},\theta^{(k+1)}\bigr)$，并更新下界  

$$
\mathrm{LB}\;\leftarrow\;c^{\top}x^{(k+1)}+\theta^{(k+1)},\qquad
k\;\leftarrow\;k+1
$$


### 4.3 终止
当 $\mathrm{UB}-\mathrm{LB}\le\varepsilon$ 时，  

$$
x^{(k)},\;y^{\star}\!\bigl(x^{(k)}\bigr)
$$

即为原问题的近似最优解。


## 5 案例：固定费用设施选址

### 5.1 问题描述  
- **目标**：给定 3 个候选仓库、4 个客户需求，决定哪些仓库投入使用，并将货物从开站仓库分配给客户，**最小化**总成本，包括：  
  - 固定开站费  
  - 年度运输费  
- **参数**  
  - 固定费用 $F_i$、容量 $C_i$、单位运费 $c_{ij}$  
  - 客户需求 $d_j$  
- **决策变量**  
  - $x_i \in \{0,1\}$：仓库 $i$ 是否启用  
  - $y_{ij} \ge 0$：从仓库 $i$ 运往客户 $j$ 的量  


### 5.2 原问题数学模型

#### 5.2.1 决策变量  
- $x_i \in \{0,1\}$ —— 仓库 $i$ 是否开站  
- $y_{ij} \ge 0$ —— 从仓库 $i$ 向客户 $j$ 发运的货量  

#### 5.2.2 目标函数  
$$
\min \;\; \sum_{i=1}^{3} F_i\,x_i \;+\; \sum_{i=1}^{3}\sum_{j=1}^{4} c_{ij}\,y_{ij}
$$

#### 5.2.3 约束  

1. **需求满足**  
   $$
   \sum_{i} y_{ij} \;=\; d_j ,\qquad \forall j
   $$

2. **仓库容量**  
   $$
   \sum_{j} y_{ij} \;\le\; C_i\,x_i ,\qquad \forall i
   $$  
   > 当 $x_i = 0$ 时，右端变成 0，表示禁止该仓库发货。  

3. **变量域**  
   $$
   x_i \in \{0,1\},\qquad y_{ij} \ge 0
   $$

#### 5.2.4 数据表


| 仓库 $i$ | 固定费用 $F_i$ | 容量 $C_i$ | 运费 $c_{i1}$ | 运费 $c_{i2}$ | 运费 $c_{i3}$ | 运费 $c_{i4}$ |
|:---------:|:--------------:|:------------:|:--------------:|:--------------:|:--------------:|:--------------:|
| 1 | 100 | 70 | 4 | 6 | 9 | 7 |
| 2 | 120 | 60 | 5 | 4 | 7 | 6 |
| 3 | 130 | 80 | 6 | 3 | 4 | 5 |

客户需求向量：

$$
d = (d_1,d_2,d_3,d_4) = (40,\,30,\,20,\,20)
$$


### 5.3 Benders 分解思路  

- **主问题**：  
  仅包含整数决策变量 $x_i \in \{0,1\}$，表示仓库是否启用；  
  引入辅助变量 $\theta$ 作为子问题运输成本的估计。  
  主问题目标为：  
  $$
  \min\; \sum_i F_i x_i + \theta
  $$
  主问题本身不考虑运输细节，通过逐步加入 Benders 割（可行性割 / 最优性割）逼近原问题。

- **子问题**：  
  固定当前主问题给定的仓库启用方案 $\bar{x}$，解决一个运输线性规划，计算最小运输成本：  
  $$
  \min\; \sum_{i,j} c_{ij} y_{ij} \quad \text{s.t. 需求满足 + 容量不超}
  $$  
  - 若子问题 **可行有界**：生成 **最优性割**，用于下界估计 $\theta$  
  - 若子问题 **不可行**（如客户需求无法全部满足）：生成 **可行性割**，排除当前不可行的 $x$ 组合

通过主问题与子问题迭代交替，逐步逼近原混合整数问题的最优解。

**一些tips：**

- 在子问题的约束条件中，不等式右侧有我们松弛主问题得到的解。每次迭代将更新主问题的解，子问题的**可行域**也会不断改变，我们希望避免这种情况。因此，对子问题取对偶，将$\bar{\mathbf{x}}$移到目标函数，则对偶问题的约束条件与$\bar{\mathbf{x}}$无关，每次迭代可行域固定。另外，对偶子问题可行域只与原问题参数有关。若对偶子问题无（可行）解，则不论主问题的解取何值，子问题均无解/无界，即原问题本身无解/无界。显然，这种情况一般不会发生。因此，**对偶子问题要么可行但无界，要么有最优解**。
- **求解对偶子问题，是为了用对偶子问题的解或极射线构造可行性割或最优性割。**
- 子问题建模有**两种思路**：一是**求解对偶子问题**，可以直接解出对偶变量（或极射线）并构造割，但如果需要知道子问题的变量值，需要对对偶子问题再取对偶。二是**求解子问题**，可以直接得到子问题变量值，但是为了构造割，需要通过**Constr.Pi**获取约束的对偶变量，通过**Constr.FarkasDual**获取约束的极射线。
- 割中的$(b-Ax)$就是对偶子问题目标函数的系数项，如果加入小于约束或等式约束，目标函数系数项的形式会有所不同。

---
## 6. 代码

In [ ]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB

# ------------------------------------------------------------------
# 数据：3 个仓库、4 个客户
# ------------------------------------------------------------------
F = np.array([100, 120, 130], dtype=float)     # 仓库固定成本
A = np.array([70, 60,  80],  dtype=float)      # 仓库产能
b = np.array([40, 30,  20, 20], dtype=float)   # 客户需求

d = np.array([[4, 6, 9, 7],                    # 运输成本 (3 × 4)
              [5, 4, 7, 6],
              [6, 3, 4, 5]], dtype=float)

warehouses = range(3)
customers  = range(4)
total_demand = b.sum()

# ------------------------------------------------------------------
# 主问题：x_i（二进制开仓变量） + θ（递归成本）
# ------------------------------------------------------------------
master = gp.Model("master")
master.Params.LogToConsole = 0

x = master.addVars(warehouses, vtype=GRB.BINARY, name="x")
theta = master.addVar(lb=0, name="theta")

# 目标：固定成本 + θ
master.setObjective(gp.quicksum(F[i] * x[i] for i in warehouses) + theta,
                    GRB.MINIMIZE)

# *可选* 约束：保证子问题可行（若想测试可行割，可注释掉此行）
# master.addConstr(gp.quicksum(A[i] * x[i] for i in warehouses) >= total_demand,
#                  name="basic_capacity")

# ------------------------------------------------------------------
# Benders 主循环
# ------------------------------------------------------------------
UB, LB = float("inf"), -float("inf")
EPS, MAX_ITER = 1e-6, 100

x_current = np.zeros(len(warehouses))            # 初始全开仓
best_x, best_y = None, None

for it in range(1, MAX_ITER + 1):
    print(f"\n=== 迭代 {it} ===")

    # ---------------- 子问题 (运输) ----------------
    sub = gp.Model("sub")
    sub.Params.LogToConsole = 0

    y = sub.addVars(warehouses, customers, lb=0, name="y")
    sub.setObjective(gp.quicksum(d[i, j] * y[i, j]
                                 for i in warehouses for j in customers),
                     GRB.MINIMIZE)

    # 产能 ≤ A_i * x̄_i
    for i in warehouses:
        sub.addConstr(gp.quicksum(y[i, j] for j in customers)
                      <= A[i] * x_current[i], name=f"cap_{i}")

    # 需求 = b_j
    for j in customers:
        sub.addConstr(gp.quicksum(y[i, j] for i in warehouses)
                      == b[j], name=f"dem_{j}")

    sub.optimize()

    # ---------- 1) 子问题可行 → 最优割 ----------
    if sub.Status == GRB.OPTIMAL:
        transp = sub.ObjVal
        total  = F @ x_current + transp

        if total < UB:
            UB, best_x = total, x_current.copy()
            best_y = np.array([[y[i, j].X for j in customers] for i in warehouses])

        print(f"子问题最优 | 运输 {transp:.2f} | 总 {total:.2f} | UB {UB:.2f}")

        cap_dual = np.array([sub.getConstrByName(f"cap_{i}").Pi for i in warehouses])
        dem_dual = np.array([sub.getConstrByName(f"dem_{j}").Pi for j in customers])

        rhs_const = dem_dual @ b                               # π_dem ᵀ b
        rhs_xpart = gp.quicksum(A[i] * cap_dual[i] * x[i]      # Σ A_i π_cap x_i
                                for i in warehouses)

        master.addConstr(theta >= rhs_const + rhs_xpart,
                         name=f"optcut_{it}")

    # ---------- 2) 子问题不可行 → 可行割 ----------
    elif sub.Status == GRB.INFEASIBLE:
        print("子问题不可行，生成可行割")
        sub.Params.InfUnbdInfo = 1
        sub.optimize()                                         # 触发 Farkas 信息

        cap_ray = np.array([sub.getConstrByName(f"cap_{i}").FarkasDual
                            for i in warehouses])
        dem_ray = np.array([sub.getConstrByName(f"dem_{j}").FarkasDual
                            for j in customers])

        expr = (dem_ray @ b) + gp.quicksum(A[i] * cap_ray[i] * x[i]
                                           for i in warehouses)

        # 计算当前 x̄ 的值以判定不等式方向
        viol = (dem_ray @ b) + np.sum(A * cap_ray * x_current)

        if viol > 0:
            master.addConstr(expr <= 0, name=f"feascut_{it}")  # πᵀ rhs > 0
            sign = "<="
        else:
            master.addConstr(expr >= 0, name=f"feascut_{it}")  # πᵀ rhs < 0
            sign = ">="
        print(f"添加可行割: πᵀrhs({viol:+.2f}) {sign} 0")

    else:
        raise RuntimeError(f"子问题状态异常：{sub.Status}")

    # ---------------- 求解主问题 ----------------
    master.optimize()
    if master.Status != GRB.OPTIMAL:
        raise RuntimeError("主问题求解失败")

    LB = master.ObjVal
    x_current = np.array([x[i].X for i in warehouses])
    gap = UB - LB
    print(f"主问题 | LB {LB:.2f} | UB {UB:.2f} | Gap {gap:.6f}")

    if gap <= EPS:
        print("\n*** 收敛 ***")
        break

# ------------------------------------------------------------------
# 输出结果
# ------------------------------------------------------------------
print("\n========== 最终结果 ==========")
print(f"最优目标值: {UB:.2f}\n")

print("仓库启用情况:")
for i in warehouses:
    print(f"  仓库 {i+1}: {'启用' if best_x[i] > 0.5 else '关闭'}")

print("\n运输矩阵 (仓库行 × 客户列):")
print(best_y)

print(f"\n固定成本 : {F @ best_x:.2f}")
print(f"运输成本 : {np.sum(d * best_y):.2f}")
print(f"总成本   : {UB:.2f}")


Set parameter LogToConsole to value 0

=== 迭代 1 ===
Set parameter LogToConsole to value 0
子问题不可行，生成可行割
添加可行割: πᵀrhs(-110.00) >= 0
主问题 | LB 220.00 | UB inf | Gap inf

=== 迭代 2 ===
Set parameter LogToConsole to value 0
子问题最优 | 运输 550.00 | 总 770.00 | UB 770.00
主问题 | LB 480.00 | UB 770.00 | Gap 290.000000

=== 迭代 3 ===
Set parameter LogToConsole to value 0
子问题最优 | 运输 470.00 | 总 720.00 | UB 720.00
主问题 | LB 630.00 | UB 720.00 | Gap 90.000000

=== 迭代 4 ===
Set parameter LogToConsole to value 0
子问题最优 | 运输 430.00 | 总 660.00 | UB 660.00
主问题 | LB 660.00 | UB 660.00 | Gap 0.000000

*** 收敛 ***

========== 最终结果 ==========
最优目标值: 660.00

仓库启用情况:
  仓库 1: 启用
  仓库 2: 关闭
  仓库 3: 启用

运输矩阵 (仓库行 × 客户列):
[[40.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0. 30. 20. 20.]]

固定成本 : 230.00
运输成本 : 430.00
总成本   : 660.00
